In [1]:
import geemap
import ee
geemap.set_proxy(port=10809)
ee.Authenticate()
ee.Initialize(project='ee-mrwurenzhe')

# 定义时空范围
start_date = '2015-01-01'
end_date = '2024-12-31'
region = ee.Geometry.Rectangle([70, 10, 140, 60])  # 东亚范围（min_lon, min_lat, max_lon, max_lat）
era5_monthly = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR").filterDate(start_date, end_date).select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])#

# 计算时间平均（所有月份的平均）
u_mean = era5_monthly.select('u_component_of_wind_10m').mean()
v_mean = era5_monthly.select('v_component_of_wind_10m').mean()

# 可视化参数 - 分别设置
u_vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}

v_vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}

# 在地图中显示
Map = geemap.Map()
Map.centerObject(region, 4)
Map.addLayer(u_mean, u_vis_params, 'ERA5 Mean U Wind (2023)')
Map.addLayer(v_mean, v_vis_params, 'ERA5 Mean V Wind (2023)')
Map.addLayer(region, {'color': 'yellow'}, 'ROI')
Map

Map(center=[35.972644113075546, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [ ]:
import geemap
import ee
import numpy as np
import math

# 设置代理（根据你的网络环境）
geemap.set_proxy(port=10809)

# 认证和初始化
ee.Authenticate()
ee.Initialize(project='ee-mrwurenzhe')

# 定义时空范围
start_date = '2024-06-01'
end_date = '2024-9-30'
region = ee.Geometry.Rectangle([70, 10, 140, 60])  # 东亚范围（min_lon, min_lat, max_lon, max_lat）

# 获取 ERA5 月平均数据
era5_monthly = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
    .filterDate(start_date, end_date) \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])

# 计算时间平均（所有月份的平均）
u_mean = era5_monthly.select('u_component_of_wind_10m').mean()
v_mean = era5_monthly.select('v_component_of_wind_10m').mean()

# 计算风速和风向
def calculate_wind_properties(image_u, image_v):
    # 风速 = sqrt(u^2 + v^2)
    wind_speed = image_u.pow(2).add(image_v.pow(2)).sqrt().rename('wind_speed')
    
    # 风向（弧度）= atan2(v, u)，转换为气象风向（单位：度）
    wind_direction = image_v.atan2(image_u) \
        .multiply(180 / math.pi) \
        .add(180) \
        .rename('wind_direction')  # 转换为 0-360 度，风的来源方向
    
    return wind_speed, wind_direction

# 计算风速和风向
wind_speed, wind_direction = calculate_wind_properties(u_mean, v_mean)

# 可视化参数
wind_speed_vis_params = {
    'min': 0,
    'max': 15,
    'palette': ['blue', 'green', 'yellow', 'red']
}

wind_direction_vis_params = {
    'min': 0,
    'max': 360,
    'palette': ['purple', 'blue', 'green', 'yellow', 'red']
}

u_vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}

v_vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}

# 创建矢量场（修复投影问题）
def create_vector_field(u, v, scale=0.1):
    # 设置默认投影（使用 ERA5 的原始投影，WGS84，约 0.1 度分辨率）
    projection = ee.Projection('EPSG:4326').atScale(11132)  # ERA5 分辨率约为 0.1 度，约 11.1km
    u = u.setDefaultProjection(projection)
    v = v.setDefaultProjection(projection)
    
    # 降采样以减少计算量
    u_sampled = u.reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1024) \
                 .reproject(crs='EPSG:4326', scale=11132)
    v_sampled = v.reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1024) \
                 .reproject(crs='EPSG:4326', scale=11132)
    
    return u_sampled, v_sampled

# 创建地图
Map = geemap.Map()
Map.centerObject(region, 4)

# 添加风速和风向图层
Map.addLayer(wind_speed, wind_speed_vis_params, 'ERA5 Mean Wind Speed (2015-2024)')
Map.addLayer(wind_direction, wind_direction_vis_params, 'ERA5 Mean Wind Direction (2015-2024)')
Map.addLayer(region, {'color': 'yellow'}, 'ROI')

# 添加降采样的 u 和 v 分量
u_sampled, v_sampled = create_vector_field(u_mean, v_mean)
Map.addLayer(u_sampled, u_vis_params, 'Sampled U Wind')
Map.addLayer(v_sampled, v_vis_params, 'Sampled V Wind')

# 显示地图
Map

Map(center=[35.972644113075546, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [13]:
import geemap

# Define output path
output_file = "ERA5_Mean_Wind_Direction_2024q3.tif"
geemap.download_ee_image(
    image=wind_direction,
    filename=output_file,
    region=region,
    scale=11132,  # ERA5-Land resolution
    crs='EPSG:4326'
)


output_file = "ERA5_Mean_Wind_u_mean_2024q3.tif"
geemap.download_ee_image(
    image=u_mean,
    filename=output_file,
    region=region,
    scale=11132,  # ERA5-Land resolution
    crs='EPSG:4326'
)


output_file = "ERA5_Mean_Wind_v_mean_2024q3.tif"
geemap.download_ee_image(
    image=v_mean,
    filename=output_file,
    region=region,
    scale=11132,  # ERA5-Land resolution
    crs='EPSG:4326'
)

ERA5_Mean_Wind_Direction_2024q3.tif: |          | 0.00/3.07M (raw) [  0.0%] in 00:00 (eta:     ?)

ERA5_Mean_Wind_u_mean_2024q3.tif: |          | 0.00/3.07M (raw) [  0.0%] in 00:00 (eta:     ?)

ERA5_Mean_Wind_v_mean_2024q3.tif: |          | 0.00/3.07M (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
import geemap
import ee
import json

# 设置代理（根据你的网络环境调整端口）
geemap.set_proxy(port=10809)

# 认证并初始化 Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-mrwurenzhe')

# 定义时空范围
start_date = '2023-01-01'
end_date = '2023-12-31'
region = ee.Geometry.Rectangle([70, 10, 140, 60])  # 东亚范围（min_lon, min_lat, max_lon, max_lat）

# 加载 ERA5-Land 月度数据并选择风分量
era5_monthly = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR").filterDate(start_date, end_date).select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])

# 计算全年平均风分量
u_mean = era5_monthly.select('u_component_of_wind_10m').mean()
v_mean = era5_monthly.select('v_component_of_wind_10m').mean()

# 可视化参数 - u 和 v 分量栅格
u_vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}
v_vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}

# 创建规则网格点用于采样（每 2 度一个点）
grid_spacing = 2  # 网格间距（度）
grid = ee.FeatureCollection(
    ee.List.sequence(70, 140, grid_spacing).map(
        lambda lon: ee.List.sequence(10, 60, grid_spacing).map(
            lambda lat: ee.Feature(
                ee.Geometry.Point([lon, lat]),
                {'lon': lon, 'lat': lat}
            )
        )
    ).flatten()
)

# 在网格点上采样 u 和 v 分量，处理空值
def sample_wind(point):
    u_val = u_mean.reduceRegion(ee.Reducer.mean(), point.geometry(), 1000).get('u_component_of_wind_10m')
    v_val = v_mean.reduceRegion(ee.Reducer.mean(), point.geometry(), 1000).get('v_component_of_wind_10m')
    # 使用 ee.Algorithms.If 处理空值
    u_val = ee.Algorithms.If(ee.Algorithms.IsEqual(u_val, None), 0, u_val)
    v_val = ee.Algorithms.If(ee.Algorithms.IsEqual(v_val, None), 0, v_val)
    return ee.Feature(point.geometry(), {
        'lon': point.get('lon'),
        'lat': point.get('lat'),
        'u': ee.Number(u_val).toFloat(),
        'v': ee.Number(v_val).toFloat()
    })

wind_samples = grid.map(sample_wind)

# 创建 geemap 地图
Map = geemap.Map()
Map.centerObject(region, 4)

# 添加 u 和 v 分量栅格图层
Map.addLayer(u_mean, u_vis_params, 'ERA5 Mean U Wind (2023)')
Map.addLayer(v_mean, v_vis_params, 'ERA5 Mean V Wind (2023)')
Map.addLayer(region, {'color': 'yellow'}, 'ROI')

# 获取风场样本数据（客户端，限制点数以提高性能）
wind_samples_list = wind_samples.limit(100).getInfo()['features']
lons, lats, u_vals, v_vals = [], [], [], []
for feature in wind_samples_list:
    lon = feature['properties']['lon']
    lat = feature['properties']['lat']
    u = feature['properties']['u']
    v = feature['properties']['v']
    # 仅添加非 None 的值
    if u is not None and v is not None:
        lons.append(lon)
        lats.append(lat)
        u_vals.append(u)
        v_vals.append(v)

# 调试：打印有效点数
print(f"有效采样点数: {len(lons)}")

# 创建 GeoJSON 表示风场矢量（线段）
geojson_features = []
for lon, lat, u, v in zip(lons, lats, u_vals, v_vals):
    end_lon = lon + u * 0.05  # 调整矢量长度
    end_lat = lat + v * 0.05
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': [[lon, lat], [end_lon, end_lat]]
        },
        'properties': {
            'u': u,
            'v': v,
            'popup': f'U: {u:.2f}, V: {v:.2f}'
        }
    }
    geojson_features.append(feature)

geojson_data = {
    'type': 'FeatureCollection',
    'features': geojson_features
}

# 添加 GeoJSON 到地图
Map.add_geojson(
    geojson_data,
    layer_name='Wind Vectors (2023)',
    style={'color': 'red', 'weight': 2},
    hover_style={'color': 'orange'},
    info_mode='on_click'  # 显示 popup
)

# 显示地图
Map

EEException: Image.reduceResolution: The input to reduceResolution does not have a valid default projection. Use setDefaultProjection() first.